## 讀取資料

In [1]:
import pandas
dfs = pandas.read_html('https://rate.bot.com.tw/xrt/quote/l6m/USD')

In [5]:
df = dfs[0].iloc[:,0:4]

In [6]:
df.columns = ['掛牌日期', '幣別', '現金匯率 - 本行買入','現金匯率 - 本行賣出']

In [8]:
df.head()

,掛牌日期,幣別,現金匯率 - 本行買入,現金匯率 - 本行賣出
0,2019/07/15,美金 (USD),30.640,31.310
1,2019/07/12,美金 (USD),30.670,31.340
2,2019/07/11,美金 (USD),30.620,31.290
3,2019/07/10,美金 (USD),30.735,31.405
4,2019/07/09,美金 (USD),30.765,31.435


## 用Python 做按鍵精靈

In [12]:
import pyautogui
#pyautogui.position()
pyautogui.click(1065, 45, duration=3)

## 資料清理

In [13]:
df.head()

,掛牌日期,幣別,現金匯率 - 本行買入,現金匯率 - 本行賣出
0,2019/07/15,美金 (USD),30.640,31.310
1,2019/07/12,美金 (USD),30.670,31.340
2,2019/07/11,美金 (USD),30.620,31.290
3,2019/07/10,美金 (USD),30.735,31.405
4,2019/07/09,美金 (USD),30.765,31.435


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 4 columns):
掛牌日期           120 non-null object
幣別             120 non-null object
現金匯率 - 本行買入    120 non-null float64
現金匯率 - 本行賣出    120 non-null float64
dtypes: float64(2), object(2)
memory usage: 3.8+ KB


In [15]:
import pandas
df['掛牌日期'] = pandas.to_datetime(df['掛牌日期'], format = '%Y/%m/%d')

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 4 columns):
掛牌日期           120 non-null datetime64[ns]
幣別             120 non-null object
現金匯率 - 本行買入    120 non-null float64
現金匯率 - 本行賣出    120 non-null float64
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 3.8+ KB


## 資料儲存

In [17]:
df.to_excel('rate.xlsx')

In [18]:
import sqlite3 as sqlite
with sqlite.connect('rate.sqlite') as db:
    df.to_sql('rate', con = db)

/Users/davidchiu/.pyenv/versions/3.6.2/lib/python3.6/site-packages/pandas/core/generic.py:2531: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype, method=method)


In [20]:
import sqlite3 as sqlite
with sqlite.connect('rate.sqlite') as db:
    df2  = pandas.read_sql_query('select avg("現金匯率 - 本行買入") from rate', con = db)
df2

,"avg(""現金匯率 - 本行買入"")"
0,30.595
